In [4]:
import pandas as pd

df = pd.read_csv("seance.csv")

period_R = 14
period_momentum = 4
period_disparity = 5

def calcul_indicateurs(group):
    group['Highest_High'] = group['prix_cloture_ajuste'].rolling(window=period_R).max()
    group['Lowest_Low'] = group['prix_cloture_ajuste'].rolling(window=period_R).min()

    group["%R"] = ((group["Highest_High"] - group["prix_cloture_ajuste"]) /
                   (group["Highest_High"] - group["Lowest_Low"])) * 100

    group["Momentum"] = group["prix_cloture_ajuste"] - group["prix_cloture_ajuste"].shift(period_momentum)

    #la moyenne sur 5 jours
    group["MA5"] = group["prix_cloture_ajuste"].rolling(window=period_disparity).mean()
    group["Disparity_5"] = (group["prix_cloture_ajuste"] / group["MA5"]) * 100

    group.drop(columns=['Highest_High', 'Lowest_Low', 'MA5'], inplace=True)
    return group

df = df.groupby("code_isin", group_keys=False).apply(calcul_indicateurs)
df.to_csv("seance_avec_indicateurs.csv", index=False)


<ipython-input-4-24ecf415d930>:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("code_isin", group_keys=False).apply(calcul_indicateurs)


In [4]:
import pandas as pd
import numpy as np

# Charger les données
df = pd.read_csv("seance.csv")


period_R = 14
period_momentum = 4
period_disparity = 5
period_volatility = 30
period_ma = 14


risk_free_rate = 0.10

def calcul_indicateurs(group):
    #%R
    group['Highest_High'] = group['prix_cloture_ajuste'].rolling(window=period_R).max()
    group['Lowest_Low'] = group['prix_cloture_ajuste'].rolling(window=period_R).min()
    group["%R"] = ((group["Highest_High"] - group["prix_cloture_ajuste"]) /
                   (group["Highest_High"] - group["Lowest_Low"])) * 100

    # Momentum
    group["Momentum"] = group["prix_cloture_ajuste"] - group["prix_cloture_ajuste"].shift(period_momentum)

    #Disparity 5
    group["MA5"] = group["prix_cloture_ajuste"].rolling(window=period_disparity).mean()
    group["Disparity_5"] = (group["prix_cloture_ajuste"] / group["MA5"]) * 100

    #MA
    group["MA"] = group["prix_cloture_ajuste"].rolling(window=period_ma).mean()

    # Rendements quotidiens (pour Sharpe Ratio et Volatilité)
    group["Returns"] = group["prix_cloture_ajuste"].pct_change()

    # Sharpe Ratio (moyenne des rendements / volatilité)
    group["Sharpe_Ratio"] = (group["Returns"].rolling(period_volatility).mean() - risk_free_rate) / group["Returns"].rolling(period_volatility).std()
    # Remplacer les valeurs infinies ou NaN par   (ou NaN selon ton besoin)
    group["Sharpe_Ratio"] = group["Sharpe_Ratio"].replace([np.inf, -np.inf], np.nan)
    # Volatilité
    group["Volatility"] = group["Returns"].rolling(period_volatility).std()

    #MCT
    group["MCT"] = (group["prix_cloture_ajuste"] * group["qte_echangee"]).cumsum() / group["qte_echangee"].cumsum()

    group.drop(columns=['Highest_High', 'Lowest_Low', 'MA5', 'Returns'], inplace=True)
    return group


df = df.groupby("code_isin", group_keys=False).apply(calcul_indicateurs)
df.to_csv("seance_avec_indicateurs.csv", index=False)


<ipython-input-4-295c6b0e8b78>:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("code_isin", group_keys=False).apply(calcul_indicateurs)
